## Backbone

In [1]:
import io
import zipfile
import numpy as np
import pandas as pd
from typing import List, Optional
import time
import asyncio
import async_timeout
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import StreamingResponse
import dotenv
import pendulum
from datetime import timedelta
from pydantic import BaseModel
from sqlalchemy import create_engine, asc, or_
from sqlalchemy.orm import sessionmaker
from fake_config import Settings
from fake_models import MessageSql
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.graph_objects as go

PYPLOT_PLOT = True
MATPLOTLIB_PLOT = False
MESSAGE_SQL = True
TIMEOUT_SECONDS = 5*60
MAX_DAYS_WARNING = 3

settings = Settings(_env_file=dotenv.find_dotenv())
valid_password = settings.visualizer_api_password.get_secret_value()
engine = create_engine(settings.db_url.get_secret_value())
Session = sessionmaker(bind=engine)

app = FastAPI()

def to_fahrenheit(t):
    return t*9/5+32

# ------------------------------
# Request types
# ------------------------------

class DataRequest(BaseModel):
    house_alias: str
    password: str
    start_ms: int
    end_ms: int
    selected_channels: List[str]
    ip_address: str
    user_agent: str
    timezone: str
    continue_option: Optional[bool] = False

In [2]:
def get_data(request):

    if request.password != valid_password:
        with open('failed_logins.log', 'a') as log_file:
            log_entry = f"{pendulum.now()} - Failed login from {request.ip_address} with password: {request.password}\n"
            log_entry += f"Timezone '{request.timezone}', device: {request.user_agent}\n\n"
            log_file.write(log_entry)
        return {
            "success": False, 
            "message": "Wrong password.", 
            "reload":True
            }, 0, 0, 0, 0
    
    if request.house_alias == '':
        return {
            "success": False, 
            "message": "Please enter a house alias.", 
            "reload": True
            }, 0, 0, 0, 0
    
    if not request.continue_option:
        if (request.end_ms - request.start_ms)/1000/60/60/24 > MAX_DAYS_WARNING:
            warning_message = f"That's a lot of data! This could take a while, "
            warning_message += f"and eventually trigger a timeout (after {int(TIMEOUT_SECONDS/60)} minutes). "
            warning_message += f"It might be best to get this data in several smaller requests.\n\nAre you sure you would like to continue?"
            return {
                "success": False,
                "message": warning_message, 
                "reload":False,
                "continue_option": True,
                }, 0, 0, 0, 0
    
    if MESSAGE_SQL:

        session = Session()

        messages = session.query(MessageSql).filter(
            MessageSql.from_alias.like(f'%{request.house_alias}%'),
            or_(
                MessageSql.message_type_name == "batched.readings",
                MessageSql.message_type_name == "report"
                ),
            MessageSql.message_persisted_ms >= request.start_ms,
            MessageSql.message_persisted_ms <=request.end_ms,
        ).order_by(asc(MessageSql.message_persisted_ms)).all()

        if not messages:
            return {
                "success": False, 
                "message": f"No data found for house '{request.house_alias}' in the selected timeframe.", 
                "reload":False
                }, 0, 0, 0, 0
        
        channels = {}
        for message in messages:
            for channel in message.payload['ChannelReadingList']:
                # Find the channel name
                if message.message_type_name == 'report':
                    channel_name = channel['ChannelName']
                elif message.message_type_name == 'batched.readings':
                    for dc in message.payload['DataChannelList']:
                        if dc['Id'] == channel['ChannelId']:
                            channel_name = dc['Name']
                # Store the values and times for the channel
                if channel_name not in channels:
                    channels[channel_name] = {
                        'values': channel['ValueList'],
                        'times': channel['ScadaReadTimeUnixMsList']
                    }
                else:
                    channels[channel_name]['values'].extend(channel['ValueList'])
                    channels[channel_name]['times'].extend(channel['ScadaReadTimeUnixMsList'])

    # Sort values according to time and find min/max
    min_time_ms, max_time_ms = 1e20, 0
    keys_to_delete = []
    for key in channels.keys():
        # Check the length
        if (len(channels[key]['times']) != len(channels[key]['values']) 
            or not channels[key]['times']):
            print(f"Warning: channel data is empty or has length mismatch: {key}")
            keys_to_delete.append(key)
            continue
        sorted_times_values = sorted(zip(channels[key]['times'], channels[key]['values']))
        sorted_times, sorted_values = zip(*sorted_times_values)
        if list(sorted_times)[0] < min_time_ms:
            min_time_ms = list(sorted_times)[0]
        if list(sorted_times)[-1] > max_time_ms:
            max_time_ms = list(sorted_times)[-1]
        channels[key]['values'] = list(sorted_values)
        channels[key]['times'] = pd.to_datetime(list(sorted_times), unit='ms', utc=True)
        channels[key]['times'] = channels[key]['times'].tz_convert('America/New_York')
        channels[key]['times'] = [x.replace(tzinfo=None) for x in channels[key]['times']]
    for key in keys_to_delete:
        del channels[key]

    # Find all zone channels
    zones = {}
    for channel_name in channels.keys():
        if 'zone' in channel_name and 'gw-temp' not in channel_name:
            if 'state' not in channel_name:
                channels[channel_name]['values'] = [x/1000 for x in channels[channel_name]['values']]
            zone_name = channel_name.split('-')[0]
            if zone_name not in zones:
                zones[zone_name] = [channel_name]
            else:
                zones[zone_name].append(channel_name)

    # Start and end times on plots
    min_time_ms += -(max_time_ms-min_time_ms)*0.05
    max_time_ms += (max_time_ms-min_time_ms)*0.05
    min_time_ms_dt = pd.to_datetime(min_time_ms, unit='ms', utc=True)
    max_time_ms_dt = pd.to_datetime(max_time_ms, unit='ms', utc=True)
    min_time_ms_dt = min_time_ms_dt.tz_convert('America/New_York').replace(tzinfo=None)
    max_time_ms_dt = max_time_ms_dt.tz_convert('America/New_York').replace(tzinfo=None)

    return "", channels, zones, min_time_ms_dt, max_time_ms_dt

## Fetch data

In [3]:
request = DataRequest
request.house_alias = 'fir'
request.password = ''
request.start_ms = int(pendulum.datetime(2024,11,12,1).timestamp()*1000)
request.end_ms = int(pendulum.datetime(2024,11,12,12,30).timestamp()*1000)
request.selected_channels = ['']
request.ip_address = ''
request.user_agent = ''
request.timezone = ''
request.continue_option = False

_, channels, __, ___, ____ = get_data(request)


## Data analysis

In [4]:
def get_energy(request, channels):

    channels_to_export = ['primary-flow','hp-ewt','hp-lwt',
                          'dist-flow', 'dist-swt', 'dist-rwt',
                          'buffer-depth1', 'buffer-depth2', 'buffer-depth3', 'buffer-depth4',
                          'buffer-hot-pipe', 'buffer-cold-pipe',
                          'store-flow']
    timestep_seconds = 10

    num_points = int((request.end_ms - request.start_ms) / (timestep_seconds * 1000) + 1)
    csv_times = np.linspace(request.start_ms, request.end_ms, num_points)
    csv_times_dt = pd.to_datetime(csv_times, unit='ms', utc=True)
    csv_times_dt = [x.tz_convert('America/New_York').replace(tzinfo=None) for x in csv_times_dt]
    
    csv_values = {}
    for channel in channels_to_export:
        merged = pd.merge_asof(
            pd.DataFrame({'times': csv_times_dt}),
            pd.DataFrame(channels[channel]),
            on='times',
            direction='backward'
        )
        csv_values[channel] = list(merged['values'])

    df = pd.DataFrame(csv_values)
    df['timestamps'] = csv_times_dt
    df = df[['timestamps'] + [col for col in df.columns if col != 'timestamps']]

    # Energy out of HP
    df['primary-flow'] = [x/100*3.78541/60 for x in df['primary-flow']]
    df['hp-lwt'] = [x/1000 for x in df['hp-lwt']]
    df['hp-ewt'] = [x/1000 for x in df['hp-ewt']]
    df['hp_power_kW'] = df['primary-flow'] * 4.187 * (df['hp-lwt']-df['hp-ewt'])
    # df['hp_power_kW'] = [0 if x<0 else x for x in list(df['hp_power_kW'])]
    df['hp_energy_kWh'] = df['hp_power_kW'] * timestep_seconds / 3600
    df['hp_cumulative'] = df['hp_energy_kWh'].cumsum()
    
    # Energy in Dist
    df['dist-flow'] = [x/100*3.78541/60 for x in df['dist-flow']]
    df['dist-swt'] = [x/1000 for x in df['dist-swt']]
    df['dist-rwt'] = [x/1000 for x in df['dist-rwt']]
    df['dist_power_kW'] = df['dist-flow'] * 4.187 * (df['dist-swt']-df['dist-rwt'])
    # df['dist_power_kW'] = [0 if x<0 else x for x in list(df['dist_power_kW'])]
    df['dist_energy_kWh'] = df['dist_power_kW'] * timestep_seconds / 3600
    df['dist_cumulative'] = df['dist_energy_kWh'].cumsum()

    # Energy in/out Buffer
    df['buffer_avg'] = (df['buffer-depth1'] + df['buffer-depth2'] + df['buffer-depth3'] + df['buffer-depth4'])/1000/4
    df['buffer_energy_kWh'] = 120*3.7 * 4.187/3600 * df['buffer_avg']
    df['buffer_energy_kWh'] =  df['buffer_energy_kWh'] -  list(df['buffer_energy_kWh'])[0]

    # df['buffer-flow'] = df['primary-flow'] - df['dist-flow']
    # df['buffer-hot-pipe'] = [x/1000 for x in df['buffer-hot-pipe']]
    # df['buffer-cold-pipe'] = [x/1000 for x in df['buffer-cold-pipe']]
    # df['buffer_power_kW'] = df['buffer-flow'] * 4.187 * (df['buffer-hot-pipe']-df['buffer-cold-pipe'])
    # df['buffer_energy_kWh'] = df['buffer_power_kW'] * timestep_seconds / 3600
    df['buffer_cumulative'] = df['buffer_energy_kWh'].cumsum()

    # Storage
    df['store-flow'] = [x/100*3.78541/60 for x in df['store-flow']]

    # Energy balance: Dist in + Buffer in - HP out = 0
    df['balance'] = df['hp_cumulative'] - df['dist_cumulative'] - df['buffer_cumulative']


    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=df['timestamps'], 
            y=df['hp_cumulative'], 
            mode='lines', 
            opacity=0.6,
            line=dict(color='orange', dash='solid'),
            name='HP out',
            )
        )
    fig.add_trace(
        go.Scatter(
            x=df['timestamps'], 
            y=df['dist_cumulative'], 
            mode='lines', 
            opacity=0.6,
            line=dict(color='green', dash='solid'),
            name='Dist in',
            )
        )
    fig.add_trace(
        go.Scatter(
            x=df['timestamps'], 
            y=df['buffer_cumulative'], 
            mode='lines', 
            opacity=0.6,
            line=dict(color='blue', dash='solid'),
            name='Buffer in',
            )
        )
    fig.add_trace(
        go.Scatter(
            x=df['timestamps'], 
            y=df['balance'], 
            mode='lines', 
            opacity=0.8,
            line=dict(color='red', dash='solid'),
            name='Energy out-in',
            )
        )
    fig.add_trace(
        go.Scatter(
            x=df['timestamps'], 
            y=df['dist_power_kW'], 
            mode='lines', 
            opacity=0.6,
            line=dict(color='purple', dash='solid'),
            name='Dist power',
            yaxis='y2',
            visible='legendonly'
            )
        )
    fig.add_trace(
        go.Scatter(
            x=df['timestamps'], 
            y=df['hp_power_kW'], 
            mode='lines', 
            opacity=0.6,
            line=dict(color='purple', dash='solid'),
            name='HP power',
            yaxis='y2',
            visible='legendonly'
            )
        )
    # fig.add_trace(
    #     go.Scatter(
    #         x=df['timestamps'], 
    #         y=df['buffer_power_kW'], 
    #         mode='lines', 
    #         opacity=0.6,
    #         line=dict(color='purple', dash='solid'),
    #         name='Buffer power',
    #         yaxis='y2',
    #         visible='legendonly'
    #         )
    #     )
    fig.update_layout(
        plot_bgcolor='white',
        paper_bgcolor='white',
        yaxis=dict(
            zeroline=False,
            showgrid=True, 
            gridwidth=1, 
            gridcolor='LightGray'
            ),
        yaxis2=dict(
            mirror=True,
            zeroline=False,
            showline=False,
            showgrid=False,
            overlaying='y', 
            side='right'
            ),
    )
    fig.show()

    # return energy_out_HP, energy_in_dist, energy_in_buffer

get_energy(request, channels)